In [ ]:
''' !git clone https://github.com/Han-JaeHoon/QRNN-for-Sequential-Classification.git
!pip install pykan
!pip install pennylane
%cd QRNN-for-Sequential-Classification '''

In [1]:
# Quantum
import pennylane as qml
# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# Numpy, Pandas
import numpy as np
import pandas as pd
# Layer
from kan import KAN
from RNN_block import RNN_block
# Data processing
from fucntions import data_seq, train_seq
from sklearn.preprocessing import MinMaxScaler
# Quantum User-Def Classes
from utils import my_utils
from NQE_class import NQE
from NQE_train_class import NQE_Train

### Settings

## Data uploading

In [2]:
locations = ['Adelaide', 'Albany', 'Albury', 'AliceSprings', 'BadgerysCreek', 'Ballarat', 'Bendigo', 'Brisbane', 'Cairns', 'Canberra', 'Cobar', 'CoffsHarbour', 'Dartmoor', 'Darwin', 'GoldCoast', 'Hobart', 'Katherine', 'Launceston', 'Melbourne', 'MelbourneAirport', 'Mildura', 'Moree', 'MountGambier', 'MountGinini', 'Newcastle', 'Nhil', 'NorahHead', 'NorfolkIsland', 'Nuriootpa', 'PearceRAAF', 'Penrith', 'Perth', 'PerthAirport', 'Portland', 'Richmond', 'Sale', 'SalmonGums', 'Sydney', 'SydneyAirport', 'Townsville', 'Tuggeranong', 'Uluru', 'WaggaWagga', 'Walpole', 'Watsonia', 'Williamtown', 'Witchcliffe', 'Wollongong', 'Woomera']

In [3]:
nqe_train = 200

In [4]:
train_data_dict = dict()
label_data_dict = dict()
for e in locations:
    print(e)
    train_df = pd.read_csv("./data/train_data_" + e + ".csv")
    label_df = pd.read_csv("./data/label_data_" + e + ".csv")
    train_data_dict[e] = torch.tensor(train_df[["MinTemp","MaxTemp","Rainfall","Humidity3pm","Pressure3pm"]].to_numpy()[:nqe_train]).to(torch.float)
    label_data_dict[e] = torch.tensor(label_df['RainTomorrow'].to_numpy()[:nqe_train]).to(torch.float)


Adelaide
Albany
Albury
AliceSprings
BadgerysCreek
Ballarat
Bendigo
Brisbane
Cairns
Canberra
Cobar
CoffsHarbour
Dartmoor
Darwin
GoldCoast
Hobart
Katherine
Launceston
Melbourne
MelbourneAirport
Mildura
Moree
MountGambier
MountGinini
Newcastle
Nhil
NorahHead
NorfolkIsland
Nuriootpa
PearceRAAF
Penrith
Perth
PerthAirport
Portland
Richmond
Sale
SalmonGums
Sydney
SydneyAirport
Townsville
Tuggeranong
Uluru
WaggaWagga
Walpole
Watsonia
Williamtown
Witchcliffe
Wollongong
Woomera


In [5]:
nqe_x_train = train_data_dict[locations[0]]
nqe_y_train = label_data_dict[locations[0]]


non_zero = (nqe_y_train != 0).nonzero()

nqe_x_train = torch.squeeze(train_data_dict[locations[0]][non_zero])
nqe_y_train = torch.squeeze(label_data_dict[locations[0]][non_zero])



import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(nqe_x_train , nqe_y_train, test_size=0.2, random_state=42)

# 데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Tensor로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11388\1777754041.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_11388\1777754041.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [6]:
nqe_train_list = []
nqe_train_label_list = []
nqe_test_list = []
nqe_test_label_list = []

for i in range(len(X_train_tensor)):
    nqe_train_data = torch.stack([X_train_tensor, torch.concat([X_train_tensor[(i + 1) : ], X_train_tensor[ : (i + 1)]])])
    nqe_train_list.append(nqe_train_data)
    nqe_label_data = torch.stack([y_train_tensor,torch.concat([y_train_tensor[(i + 1) :],y_train_tensor[: (i + 1)]])])
    nqe_train_label_list.append(nqe_label_data)
  
for i in range(len(X_test_tensor)):
    nqe_test_data = torch.stack([X_test_tensor, torch.concat([X_test_tensor[(i + 1) : ], X_test_tensor[ : (i + 1)]])])
    nqe_test_list.append(nqe_test_data)
    nqe_label_data = torch.stack([y_test_tensor,torch.concat([y_test_tensor[(i + 1) :],y_test_tensor[: (i + 1)]])])
    nqe_test_label_list.append(nqe_label_data)

nqe_train_data = torch.concat(nqe_train_list, dim = 1)
nqe_train_label = torch.concat(nqe_train_label_list, dim = 1)
nqe_test_data = torch.concat(nqe_test_list, dim = 1)
nqe_test_label = torch.concat(nqe_test_label_list, dim = 1)


print(nqe_train_data.shape)
print(nqe_train_label.shape)
print(nqe_test_data.shape)
print(nqe_test_label.shape)


nqe_train_data = nqe_train_data.permute(1,0,2)
nqe_test_data = nqe_test_data.permute(1,0,2)
nqe_train_label = nqe_train_label.permute(1,0)
nqe_test_label = nqe_test_label.permute(1,0)

print(nqe_train_data.shape)
print(nqe_train_label.shape)
print(nqe_test_data.shape)
print(nqe_test_label.shape)


from torch.utils.data import DataLoader, TensorDataset
train_dataset = TensorDataset(nqe_train_data, nqe_train_label)
test_dataset = TensorDataset(nqe_test_data, nqe_test_label)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

torch.Size([2, 21025, 5])
torch.Size([2, 21025])
torch.Size([2, 1369, 5])
torch.Size([2, 1369])
torch.Size([21025, 2, 5])
torch.Size([21025, 2])
torch.Size([1369, 2, 5])
torch.Size([1369, 2])


In [7]:
nqe_train_label

tensor([[-1., -1.],
        [-1., -1.],
        [-1., -1.],
        ...,
        [-1., -1.],
        [-1., -1.],
        [-1., -1.]])

## NQE Train

In [8]:
import sys
def criterion(pred, label):
    '''
        pred : inner product of two states
        label : label data
    '''
    # print(pred.shape)
    # print(label.shape)
    loss = torch.sum(((pred) - 0.5 * (label[:, 0] * label[:, 1] + 1)) ** 2 ) / len(pred)
    return loss

def accuarcy(pred, label):
    '''
        pred : inner product of two states
        label : label data
    '''
    acc = torch.sum((torch.round(pred) == torch.round(0.5 * (label[:, 0] * label[:, 1] + 1)))) / len(pred)
    return acc

In [10]:
# nqe1.load_state_dict(torch.load("./models/nqe_fc_loss284.pth"))

In [11]:
n_qu = 5 # number of features
nqe1 = NQE(n_qu, 'FC')
nqe_train1 = NQE_Train(nqe1, criterion, train_loader,test_loader,[accuarcy])

In [12]:
trained_nqe = nqe_train1.train(15)

 loss : 0.01354 metric0 : 1.00000

 train_loss : 0.01354 test_loss : 0.28805 train_metric : 1.00000 test_metric : 0.60263

 loss : 0.00632 metric0 : 1.00000

 train_loss : 0.00632 test_loss : 0.32027 train_metric : 1.00000 test_metric : 0.59240

 loss : 0.01885 metric0 : 0.96970

 train_loss : 0.01885 test_loss : 0.32144 train_metric : 0.96970 test_metric : 0.55296

 loss : 0.00057 metric0 : 1.00000

 train_loss : 0.00057 test_loss : 0.36111 train_metric : 1.00000 test_metric : 0.55004

 loss : 0.03067 metric0 : 0.96970

 train_loss : 0.03067 test_loss : 0.35832 train_metric : 0.96970 test_metric : 0.54273

 loss : 0.00026 metric0 : 1.00000

 train_loss : 0.00026 test_loss : 0.36900 train_metric : 1.00000 test_metric : 0.54565

 loss : 0.00016 metric0 : 1.00000

 train_loss : 0.00016 test_loss : 0.37230 train_metric : 1.00000 test_metric : 0.55004

 loss : 0.03042 metric0 : 0.96970

 train_loss : 0.03042 test_loss : 0.40378 train_metric : 0.96970 test_metric : 0.50183

 loss : 0.00003 

---

In [13]:
torch.save(trained_nqe.state_dict(), "./models/nqe_fc_loss323.pth")

In [17]:
nqe2 = NQE(n_qu, 'KAN')
nqe_train2 = NQE_Train(nqe2, criterion, train_loader, test_loader, [accuarcy])

In [18]:
trained_nqe2 = nqe_train2.train(10)

 loss : 0.17146 metric0 : 0.78788

 train_loss : 0.17146 test_loss : 0.24324 train_metric : 0.78788 test_metric : 0.64207

 loss : 0.16672 metric0 : 0.78788

 train_loss : 0.16672 test_loss : 0.24293 train_metric : 0.78788 test_metric : 0.65230

 loss : 0.11358 metric0 : 0.81818

 train_loss : 0.11358 test_loss : 0.25046 train_metric : 0.81818 test_metric : 0.64354

 loss : 0.10798 metric0 : 0.81818

 train_loss : 0.10798 test_loss : 0.26714 train_metric : 0.81818 test_metric : 0.57633

 loss : 0.15382 metric0 : 0.81818

 train_loss : 0.15382 test_loss : 0.27411 train_metric : 0.81818 test_metric : 0.56757

 loss : 0.14362 metric0 : 0.78788

 train_loss : 0.14362 test_loss : 0.30407 train_metric : 0.78788 test_metric : 0.60847

 loss : 0.07536 metric0 : 0.90909

 train_loss : 0.07536 test_loss : 0.31481 train_metric : 0.90909 test_metric : 0.58802

 loss : 0.04748 metric0 : 0.96970

 train_loss : 0.04748 test_loss : 0.30377 train_metric : 0.96970 test_metric : 0.58802

 loss : 0.04551 

In [19]:
torch.save(trained_nqe2.state_dict(), "./models/nqe_kan_loss291.pth")